# Predicting heart disease using classification

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning moddel capable of predicting whether or not someone has heart disease based on their medical attributes.

Approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modeling
6. Experimentation

## 1. Probelm Definitoin

In a statemetnt,
> Given clinical paramaters about a patient, can I predict whether or not they have heart disease.

## 2.  Data

The orignal data came from Cleveland data from UCI Machine Learning Repository: https://archive.ics.uci.edu/dataset/45/heart+disease

There is also a version of it available on Kaggle.
## 3. Evaluation

> If I can reach 95% accuracy at predicting whether or not a patient has heart disease,
during the proof of concept, I'll pursue the project.

## 4. Features

**Create data dictionary**

* age in years
* sex(1 = male; 0 = female)
* cpchest pain type
* trestbpsresting blood pressure (in mm HG on admission to the hospital)
* cholserum cholestoral in mg/d1
* fbs(fasting blood sugar > 120 mg/d1) (1=true; 0 = false)
* restecgresting electrocardiographic result
* thalachmaximum heart rate achieved
* exangexercise induced angina ( 1 = yes; 0 = no)
* oldpeakST depression induced by exercise relative to rest
* slopethe slope of the peak exercise ST segment
* canumber of major vessles(0-3) colored by floursopy
* thal3 = normal; 6 = fixed defect; 7= reverasble defect
* target = 1 or 0

## The necessary tools

Data analysis and manipulation: pandas, Matplotlib and NumPy

In [ ]:
# Import the tools

# Regular EDA (exploratory data analysis) and plotting libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

# Models from Scikit-Learn
from sklearn.liner_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve